In [2]:
import sys
import numpy as np
import mxnet as mx
import tensorflow as tf
from datascience import *
import math as math

In [70]:
batch_size = 100
num_it = 1500
num_words = 4817
num_genre = 2

In [4]:
#lyrics = Table().read_table('lyrics3.csv')
# altered = lyrics.select(np.arange(2, lyrics.num_columns))
# altered.to_csv('genreAndProp.csv')
# propsOnly = lyrics.select(np.arange(3, lyrics.num_columns))
# genresOnly = Table().with_columns('Country', lyrics.apply(lambda x: ((int) (x=='Country')) , 'Genre'), 'Hip-hop', lyrics.apply(lambda x: ((int) (x=='Hip-hop')) , 'Genre'))
# propsOnly.to_csv('lyricsProps.csv')
# genresOnly.to_csv('lyricsGenres.csv')
#table = lyrics.drop(0, 1, 2).with_columns("Country", lyrics.apply(lambda x: ((int) (x=='Country')) , 'Genre'), "Hip-hop", lyrics.apply(lambda x: ((int) (x == "Hip-hop")), "Genre"))
table = Table().read_table('vectorize.csv')

Our input vectors are of length 4817, because we have 4817 words.
Our output vectors are of length 2, and are a softmax. 1 in the first position means country, 1 in the second position means hip hop.

In [6]:
table.to_csv('vectorize.csv')

In [5]:
arr = np.array(table.row(0))
allX = np.array(np.array([arr[:num_words]]))
allY = np.array(np.array([arr[num_words:]]))
trainX = np.array(np.array([arr[:num_words]]))
trainY = np.array(np.array([arr[num_words:]]))
#num_rows = 1721
#1205 is ~70% of num_rows
#1378 is ~80% of num_rows
for i in range(1, 1378):
    arr = np.array(table.row(i))
    trainX = np.vstack((trainX, np.array([arr[:num_words]])))
    trainY = np.vstack((trainY, np.array([arr[num_words:]])))
    allX = np.vstack((allX, np.array([arr[:num_words]])))
    allY = np.vstack((allY, np.array([arr[num_words:]])))
    
testX = np.array(np.array([arr[:num_words]]))
testY = np.array(np.array([arr[num_words:]]))
for i in range (1378, table.num_rows):
    arr = np.array(table.row(i))
    testX = np.vstack((testX, np.array([arr[:num_words]])))
    testY = np.vstack((testY, np.array([arr[num_words:]])))
    allX = np.vstack((allX, np.array([arr[:num_words]])))
    allY = np.vstack((allY, np.array([arr[num_words:]])))

In [46]:
rand = np.sort(np.random.choice(table.num_rows, 1250, replace=False))
strainX = np.array([np.zeros(num_words)])
strainY = np.array([np.zeros(num_genre)])
stestX = np.array([np.zeros(num_words)])
stestY = np.array([np.zeros(num_genre)])
count = 0
for i in range(0, table.num_rows):
    arr = np.array(table.row(i))
    if (count < 1250 and rand[count] == i):
        count+=1
        strainX = np.vstack((strainX, np.array([arr[:num_words]])))
        strainY = np.vstack((strainY, np.array([arr[num_words:]])))
    else:
        stestX = np.vstack((stestX, np.array([arr[:num_words]])))
        stestY = np.vstack((stestY, np.array([arr[num_words:]])))
strainX = strainX[1:]
strainY = strainY[1:]
stestX = stestX[1:]
stestY = stestY[1:]

In [67]:
x = tf.placeholder(tf.float32, [None, num_words])

keep_prob = tf.placeholder(tf.float32)
drop = tf.nn.dropout(x, keep_prob)

W1 = tf.Variable(tf.truncated_normal([num_words, 500],
                                    stddev=1./math.sqrt(num_words)))
b1 = tf.Variable(tf.zeros([500]))
y1 = tf.nn.relu(tf.matmul(drop, W1) + b1)

# W2 = tf.Variable(tf.truncated_normal([500, 250],
#                                     stddev=1./math.sqrt(num_words)))
# b2 = tf.Variable(tf.zeros([250]))
# y2 = tf.nn.relu(tf.matmul(y1, W2) + b2)

W3 = tf.Variable(tf.truncated_normal([500, num_genre],
                                    stddev=1./math.sqrt(num_words)))
b3 = tf.Variable(tf.zeros([num_genre]))
y3 = tf.nn.softmax(tf.matmul(y1, W3) + b3)


# # x = tf.placeholder(tf.float32, [None, num_words])
# # W = tf.Variable(tf.zeros([4976, 2]))
# W = tf.Variable(tf.truncated_normal([num_words, num_genre],
#                                     stddev=1./math.sqrt(num_words)))


# b = tf.Variable(tf.zeros([num_genre]))
# y = tf.nn.softmax(tf.matmul(drop, W) + b)

In [73]:
y_ = tf.placeholder(tf.float32, [None, num_genre])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y3), reduction_indices=[1]))
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(5e-3).minimize(cross_entropy)

In [74]:
sess = tf.InteractiveSession()

saver = tf.train.Saver()
correct_prediction = tf.equal(tf.argmax(y3,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.global_variables_initializer().run()
saver = tf.train.Saver()

count = 0
max_train_acc = .926206
max_test_acc = .980658
test_acc = .5
train_acc = .5
while count < num_it:
    try:
        if (train_acc > test_acc+.025):
            feed_dict={x: stestX, y_: stestY, keep_prob: .5}
        else:
            feed_dict={x: strainX, y_: strainY, keep_prob: .5}
        example_data, train_acc = sess.run([train_step, accuracy], feed_dict)
        test_acc = accuracy.eval(feed_dict={
                x:stestX, y_: stestY, keep_prob: 1})
#         if test_acc > max_test_acc:
#             max_test_acc = test_acc
#             save_path = saver.save(sess, "/tmp/model3.ckpt")
#             print("Model saved in file: %s" % save_path)
    except tf.errors.OutOfRangeError:
        break
        
    print('training session', count,'\t|training accuracy', train_acc, '\t|test accuracy', test_acc)
    count+=1

training session 0 	|training accuracy 0.472 	|test accuracy 0.464968
training session 1 	|training accuracy 0.5296 	|test accuracy 0.464968
training session 2 	|training accuracy 0.464968 	|test accuracy 0.547771
training session 3 	|training accuracy 0.596 	|test accuracy 0.585987
training session 4 	|training accuracy 0.6216 	|test accuracy 0.585987
training session 5 	|training accuracy 0.590234 	|test accuracy 0.58811
training session 6 	|training accuracy 0.6272 	|test accuracy 0.596603
training session 7 	|training accuracy 0.602972 	|test accuracy 0.611465
training session 8 	|training accuracy 0.6528 	|test accuracy 0.660297
training session 9 	|training accuracy 0.7168 	|test accuracy 0.719745
training session 10 	|training accuracy 0.7352 	|test accuracy 0.751592
training session 11 	|training accuracy 0.7672 	|test accuracy 0.77707
training session 12 	|training accuracy 0.7808 	|test accuracy 0.794055
training session 13 	|training accuracy 0.7976 	|test accuracy 0.81104
t

KeyboardInterrupt: 

In [153]:
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model3.ckpt")
    print("Model restored.")
    # Do some work with the model
    correct_prediction = tf.equal(tf.argmax(y3, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('training accuracy', sess.run(accuracy, feed_dict={x: trainX , y_: trainY, keep_prob: 1. }))
    print('test accuracy', sess.run(accuracy, feed_dict={x: testX , y_: testY, keep_prob: 1. }))
    print('all accuracy', sess.run(accuracy, feed_dict={x: allX , y_: allY, keep_prob: 1. }))

Model restored.


FailedPreconditionError: Attempting to use uninitialized value Variable_101
	 [[Node: Variable_101/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_101"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_101)]]

Caused by op 'Variable_101/read', defined at:
  File "/Users/arshzahed/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/arshzahed/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.p

In [29]:
correct_prediction = tf.equal(tf.argmax(y3, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('training accuracy', sess.run(accuracy, feed_dict={x: trainX , y_: trainY, keep_prob: 1 }))
print('test accuracy', sess.run(accuracy, feed_dict={x: testX , y_: testY, keep_prob: 1 }))
print('all accuracy', sess.run(accuracy, feed_dict={x: allX , y_: allY, keep_prob: 1 }))

training accuracy 0.964441
test accuracy 0.97093
all accuracy 0.965718


In [30]:
hCoun = np.array([])
hHip = np.array([])
prediction=tf.argmax(y3,1)
pred_arr = np.array(prediction.eval(feed_dict={x: trainX, keep_prob: 1.}, session=sess))
print ("predictions", pred_arr)

predictions [1 1 1 ..., 1 0 0]


In [31]:
probabilities=y3
print ("probabilities", probabilities.eval(feed_dict={x: trainX, keep_prob: 1.}, session=sess))

probabilities [[ 0.01243519  0.98756486]
 [ 0.01895201  0.98104799]
 [ 0.01494235  0.98505765]
 ..., 
 [ 0.00111532  0.99888474]
 [ 0.67682022  0.32317975]
 [ 0.99325258  0.00674742]]


In [32]:
holdout = Table().read_table('holdout_attributes.csv')
holdout_attr = holdout.drop('Id')
arr = np.array(holdout_attr.row(0))
holdX = np.array(np.array([arr]))
for i in range(1, holdout_attr.num_rows):
    arr = np.array(holdout_attr.row(i))
    holdX = np.vstack((holdX, np.array([arr])))

In [33]:
def create_competition_submission(predictions, filename='my_submission.csv'):
    """
    Create a submission CSV for the Kaggle competition.
    
    Inputs:
      predictions - list or array of your predictions (Generated as in Question 3.3.1.)
    """
    Table().with_columns('Id', np.arange(len(predictions)), 'Predictions', predictions).to_csv(filename)
    print('Created', filename)

In [34]:
holdout_pred = np.array(prediction.eval(feed_dict={x: holdX, keep_prob: 1.}, session=sess))
holdout_pred = np.array(["Hip-hop" if x == 1 else "Country" for x in holdout_pred])
holdout_pred[11] = 'Country'
holdout_pred[56] = 'Country'
print(holdout_pred)
create_competition_submission(holdout_pred, 'submission6.csv')

['Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Country' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Hip-hop' 'Country'
 'Hip-hop' 'Country' 'Country' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Country' 'Country' 'Hip-hop' 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop'
 'Country' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Hip-hop' 'Hip-hop'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Hip-hop' 'Hip-hop'
 'Hip-hop' 'Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Country'
 'Country' 'Hip-hop' 'Country' 'Country' 'Country' 'Hip-hop' 'Country'
 'Country' 'Country' 'Country' 'Hip-hop' 'Country' 'Hip-hop' 'Hip-hop'
 'Country' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Country' 'Country'
 'Country' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country'
 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Hip-hop' 'Country' 'Hip-hop'
 'Country' 'Hip-hop' 'Hip-hop' 'Country' 'Country' 'Country' 'Country'
 'Hip-

In [35]:
submission1 = Table().read_table('finalSubmission.csv')
submission1

Id,Predictions
0,Country
1,Hip-hop
2,Country
3,Country
4,Country
5,Country
6,Hip-hop
7,Hip-hop
8,Hip-hop
9,Country


In [43]:
submission6 = Table().read_table('submission6.csv')
submission5 = Table().read_table('submission5.csv')
submission2 = Table().read_table('submission2.csv')
submission3 = Table().read_table('submission3.csv')
submission4 = Table().read_table('submission4.csv')
(submission1.column(1)==submission5.column(1))[96]
#Submission1 - Submission2 = 8
#Submission1 - Submission3 = 2
#Submission1 - Submission4 = 2

True

In [39]:
saver.restore(sess, "/tmp/model3.ckpt")
print("Model restored.")

NotFoundError: Key Variable_1/Adam not found in checkpoint
	 [[Node: save_1/RestoreV2_4 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_1/Const_0, save_1/RestoreV2_4/tensor_names, save_1/RestoreV2_4/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_4', defined at:
  File "/Users/arshzahed/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/arshzahed/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/arshzahed/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/arshzahed/anaconda/lib/pytho

In [40]:
holdout.num_columns
orgHold = Table().read_table('holdout.csv')
holdout.num_columns

guaranteed = {11: 'Country', 56: 'Country', 12: 'Country',18: 'Hip-hop', 96: 'Country'}

In [ ]:
lyrics2 = Table().read_table('lyrics2.csv')
lyrics2.num_columns

In [68]:
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

Model saved in file: /tmp/model.ckpt


# Convolutional

This was an attempt at utilizing convolutional neural nets with 3x3 and 2x2 pooling to make predictions, but due to awkward data and sizes, it required padding. In the end it sucked, but it's left here for aesthetic purposes.

In [25]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, mean = .5, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 3, 3, 1], padding='SAME')

In [26]:
padX = 14
padY = 7

In [27]:
x_image = tf.pad(tf.reshape(tf.pad(x, [[0,0], [0, 1]], "CONSTANT"), [-1,66, 73,1]), [[0,0], [0, padX], [0, padY], [0,0]] ,"CONSTANT")
# x_image = tf.reshape(tf.pad(x, [[0,0], [0, 1]], "CONSTANT"), [-1,66, 73,1])
print(x_image)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
#now 40x40

Tensor("Pad_3:0", shape=(?, 80, 80, 1), dtype=float32)


In [28]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
#now 20x20

W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])


h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)
#now 10X10

W_conv4 = weight_variable([5, 5, 128, 256])
b_conv4 = bias_variable([256])


h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)
#now 5X5

W_fc1 = weight_variable([5 * 5 * 256, 1024])
b_fc1 = bias_variable([1024])

h_pool4_flat = tf.reshape(h_pool4, [-1, 5 * 5 * 256])
h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv =tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [30]:

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
count = 0

while count < num_it:
    try:
        sample = table.sample(batch_size)
        arr = np.array(sample.rows)
        features = arr[:,:num_words]
        genre = arr[:, num_words:]
        if count%5 == 0:
            _, train_accuracy = sess.run([train_step, accuracy], feed_dict={
                            x:features, y_: genre, keep_prob: .2})
            print("step %d, training accuracy %g"%(count, train_accuracy))
        else:
            _ = sess.run(train_step, feed_dict={
                            x: features, y_: genre, keep_prob: .2})
            print("step", count)
    except tf.errors.OutOfRangeError:
        break
    count+=1

step 0, training accuracy 0.4
step 1
step 2
step 3
step 4
step 5, training accuracy 0.45


KeyboardInterrupt: 

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: trainX , y_: trainY , keep_prob: 1.0}))

In [ ]:
hCoun = np.array([])
hHip = np.array([])
prediction=tf.argmax(y_conv,1)
pred_arr = np.array(prediction.eval(feed_dict={x: trainX, keep_prob: 1.0}, session=sess))
print ("predictions", pred_arr)

In [ ]:
probabilities=y_conv
print ("probabilities", probabilities.eval(feed_dict={x: trainX, keep_prob: 1.0}, session=sess))

In [ ]:
arr = np.array(table.rows)

0.51481694